In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, sys
import re

**Requirements**

The project repo contains a notebook with data collection and descriptive analysis, properly commented, and the notebook ends with a more structured and informed plan for what comes next (all the way to a plan for the presentation). These sections of the notebook should be filled in by milestone 3.

We will evaluate this milestone according to how well these previous steps (or other reasonable ones) have been done and documented, the quality of the code and its documentation, the feasibility and critical awareness of the project.

# Data collection

- That you can handle the data in its size.

## Chicago Food Inspections (CFI) dataset

In [ ]:
data = pd.read_csv('data/chicago-food-inspections/food-inspections.csv')

## Yelp Reviews (YR) dataset

## Crimes dataset

In [ ]:
crimes = pd.read_csv('data/crimes.csv')

---

# Data preprocessing

- That you understand what’s into the data (formats, distributions, missing values, correlations, etc.).
- That you considered ways to enrich, filter, transform the data according to your needs.

## Chicago Food Inspections (CFI) dataset

hugues

## Crimes dataset

eliane

## Yelp Reviews (YR) dataset

jeremie

---

# Descriptive analysis

Descriptive analysis of:

- evolution of the food inspection results
- food chains / food categories (type of cuisine, facility type etc.)
- a possible correlation of hygienic standards of the food places and their respective customer reviews
- patterns of neighborhood crime rates


## Evolution of food inspection results

camille

## Analysis of food chains and categories

hugues

## Correlation analysis  of hygienic standards of the food places and customer reviews

(jeremie)

## Patterns of neighborhood crime rates

eliane

---

# Project pipeline

- That you have updated your plan in a reasonable way, reflecting your improved knowledge after data acquaintance. In particular, discuss how your data suits your project needs and discuss the methods you’re going to use, giving their essential mathematical details in the notebook.
- That your plan for analysis and communication is now reasonable and sound, potentially discussing alternatives to your choices that you considered but dropped.

## Mid-term review

## Internal milestones

**Due by Friday, Nov 29**



**Due by Friday, Dec 06**



**Due by Friday, Dec 13**



**Due by Friday, Dec 20 - Project deadline**
- to hand in: data story in a platform of your choice (e.g., a blog post, or directly in GitHub), plus the final notebook (continuation of milestone 2).


**Due by Monday, Jan 20 - Poster presentation**



# update readme!!!